# Project 1 - Baseball  
Josh Urry

In [ ]:
# Install packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
# Import data
peopleDf = pd.read_csv('baseballdatabank-2022.2/core/People.csv')
battingDf = pd.read_csv('baseballdatabank-2022.2/core/Batting.csv')
pitchingDf = pd.read_csv('baseballdatabank-2022.2/core/Pitching.csv')
teamsDf = pd.read_csv('baseballdatabank-2022.2/core/Teams.csv')
salariesDf = pd.read_csv('baseballdatabank-2022.2/contrib/Salaries.csv')

In [ ]:
# Set fonts for charts
title = {'family':'serif','color':'black','size':20}
label = {'family':'serif','color':'black','size':15}

## Analysis 1: How does the average strikeouts change over time?

In [ ]:
pitchingDf.head()

In [ ]:
# Look at total summary stats
pitchingDf.SO.describe()

In [ ]:
plt.figure(figsize = (10,6))
sns.lineplot(data = pitchingDf, x = "yearID", y = "SO")
plt.xlabel("Year", fontdict = label)
plt.ylabel("# of Strike Outs", fontdict = label)
plt.title("Avg Strike Outs per Year", fontdict = title)

# After looking online, I found the strike zone changed multiple times throughout league history.
# Add vertical lines to see if they correspond with the trends.
# Orange represents an official "strike zone" change
# Black represents other rule changes
plt.axvline(1887, color = "grey", label = "Strike definition change")
plt.axvline(1894, color = "grey")
plt.axvline(1899, color = "grey")
plt.axvline(1901, color = "grey")
plt.axvline(1907, color = "grey")
plt.axvline(1910, color = "grey")

plt.axvline(1950, color = "orange", label = "Strike zone definition change")
plt.axvline(1963, color = "orange")
plt.axvline(1969, color = "orange")
plt.axvline(1988, color = "orange")
plt.axvline(1996, color = "orange")

plt.legend()

plt.savefig('figs/SOperYear.pdf')

Does the throwing hand affect that average?

In [ ]:
# Get throwing hand from people dataset
pitchingDf = pd.merge(pitchingDf, peopleDf[["playerID", "throws"]])

In [ ]:
# Look at throwning hand values
pitchingDf.throws.unique()

In [ ]:
# One player shows that their throwing hand is "S"
pitchingDf[pitchingDf.throws == "S"]

In [ ]:
# I can't find what S means, so I am just going to remove them from the analysis
pitchingDf = pitchingDf[pitchingDf.throws != "S"]

In [ ]:
plt.figure(figsize = (10,6))
sns.lineplot(data = pitchingDf, x = "yearID", y = "SO", hue = "throws", errorbar = None)
plt.xlabel("Year", fontdict = label)
plt.ylabel("# of Strike Outs", fontdict = label)
plt.title("Avg Strike Outs per Year by Pitching Hand", fontdict = title)
plt.savefig('figs/SOperYearH.pdf')

Look at overall averages for righties and lefties

In [ ]:
left = pitchingDf[pitchingDf.throws == "L"]
right = pitchingDf[pitchingDf.throws == "R"]
leftAvg = left.SO.mean()
rightAvg = right.SO.mean()

print("The average strike outs thrown by lefties is: ", leftAvg)
print("The average strike outs thrown by righties is: ", rightAvg)

# Display t test results
# Show t test results
display(f'Independent T-test (World Series): {scipy.stats.ttest_ind(left.SO, right.SO)}')

It looks like throwing hand doesn't make a huge difference, except for the spike around 1890. Overall, it looks like late 1800s to early 1900s players have slightly more strikeouts on average, but there is a lot of variation. Let's look at the players with the most strikeouts. I want to look at the most strike outs in a season, as well as player average. My guess is that the highest strike out counts will come from players from 1870 - 1890.

## Analysis 2: What players have the highest strikeouts in a season?

In [ ]:
# Groupby year
pitchMax = pitchingDf[["yearID", "playerID", "SO"]].groupby("playerID").max().reset_index()
pitchMax = pitchMax.sort_values(by = ["SO"], ascending = False)

In [ ]:
# Add name
pitchMax = pd.merge(pitchMax, peopleDf[["playerID", "nameFirst", "nameLast"]], on="playerID")

# Combine names
pitchMax["name"] = pitchMax.nameFirst + " " + pitchMax.nameLast

# Drop separate names
pitchMax = pitchMax.drop(["nameFirst", "nameLast"], axis = 1)

In [ ]:
# Get top 5
topSO = pitchMax.head(5)
# Get max year with name for chart
dataTypes = {'yearID': str}
topSO = topSO.astype(dataTypes)
topSO["nameYear"] = topSO.name + " " + topSO.yearID

In [ ]:
topSO

In [ ]:
plt.figure(figsize = (12,5))
sns.barplot(data = topSO, x = "nameYear", y = "SO")
plt.xlabel("Player", fontdict = label)
plt.ylabel("# of Strike Outs", fontdict = label)
plt.ylim(400, 520)
plt.title("Top 5 Strike Outs in a Season", fontdict = title)
plt.savefig('figs/MaxSO.pdf')

As suspected, the highest were in the early years. Let's see what the players averages were to see if this was just a fluke.

In [ ]:
pitchAvg = pitchingDf[["playerID", "SO"]].groupby("playerID").mean().reset_index()
# Rename SO column
pitchAvg.rename(columns = {"SO":"AvgSO"}, inplace = True)

In [ ]:
# Rename max
pitchMax.rename(columns = {"SO":"MaxSO"}, inplace = True)

In [ ]:
# Merge the average with the max
pitchMax = pd.merge(pitchMax, pitchAvg, on="playerID")
topSO = pitchMax.head()

In [ ]:
# Overlay two bar charts
plt.figure(figsize = (12,5))

sns.barplot(data = topSO, x = "name", y = "MaxSO", color = "blue", alpha = .75, label = "Max")
sns.barplot(data = topSO, x = "name", y = "AvgSO", color = "orange", label = "Avg")

plt.legend()
plt.xlabel("Player", fontdict = label)
plt.ylabel("# of Strike Outs", fontdict = label)
plt.title("Top 5 Strike Outs in a Season with Career Avg", fontdict = title)
plt.savefig('figs/MaxAvgSO.pdf')

Now let's see who has the highest averages, and if they are still in the early years

In [ ]:
pitchMax = pitchMax.sort_values(by = ["AvgSO"], ascending = False)

In [ ]:
topSO = pitchMax.head(5)
topSO

In [ ]:
plt.figure(figsize = (12,5))
sns.barplot(data = topSO, x = "name", y = "AvgSO")
plt.xlabel("Player", fontdict = label)
plt.ylabel("Avg # of Strike Outs", fontdict = label)
plt.ylim(150, 220)
plt.title("Top 5 Strike Out Avgs of All Time", fontdict = title)
plt.savefig('figs/TopAvgSO.pdf')

## Analysis 3: Do lighter players steal bases more often?
I am assuming lighter players are quicker, so they would be more likely to steal.

In [ ]:
# First, get a histogram of weights
plt.figure(figsize = (8,6))
sns.histplot(data = peopleDf, x = "weight", bins = 50)
plt.xlabel("Player Weight", fontdict = label)
plt.ylabel("Count", fontdict = label)
plt.title("Histogram of Player Weights", fontdict = title)
plt.savefig('figs/WeightHist.pdf')

In [ ]:
peopleDf.weight.describe()

In [ ]:
# First, get everyone's average stolen bases since the batting table has multiple years and there is only one weight
stealBs = battingDf[["playerID", "SB"]].groupby("playerID").mean().reset_index()
# merge weight with batting table 
stealBs = pd.merge(stealBs, peopleDf[["playerID", "weight"]], on = "playerID")

In [ ]:
# Let's just look at everyone that averaged at least 5 stolen bases a season (and drop any nans)
stealBs = stealBs[stealBs.SB >= 5].dropna()
print(len(stealBs), "players averaged at least 5 stolen base per season")

In [ ]:
plt.figure(figsize = (12,7))
sns.scatterplot(data = stealBs, x = "weight", y = "SB", s = 75)
plt.xlabel("Player Weight", fontdict = label)
plt.ylabel("Avg Stolen Bases per Season", fontdict = label)
display(f'Pearson r correlation: {scipy.stats.pearsonr(stealBs.weight, stealBs.SB)}')
plt.title("Player Weight vs Bases Stolen per Season", fontdict = title)
plt.savefig('figs/WeightVsSB.pdf')

Conversely, do heavier players get caught stealing bases more often?

In [ ]:
# First, get everyone's average stolen bases since the batting table has multiple years and there is only one weight
caught = battingDf[["playerID", "SB", "CS"]].groupby("playerID").mean().reset_index()
# merge weight with batting table 
caught = pd.merge(caught, peopleDf[["playerID", "weight"]], on = "playerID")

In [ ]:
# Let's just look at everyone that averaged at least 5 stolen bases a season (and drop any nans)
caught = caught[caught.SB >= 5].dropna()
print(len(caught), "players averaged at least 5 stolen base per season")

In [ ]:
# Look at descriptives
caught.CS.describe()

In [ ]:
plt.figure(figsize = (12,7))
sns.scatterplot(data = caught, x = "weight", y = "CS", s = 75, color = "orange")
plt.xlabel("Player Weight", fontdict = label)
plt.ylabel("Times Caught Stealing per Season", fontdict = label)
display(f'Pearson r correlation: {scipy.stats.pearsonr(caught.weight, caught.CS)}')
plt.title("Player Weight vs Times Caught Stealing per Season", fontdict = title)
plt.savefig('figs/WeightVsCS.pdf')

Nope, there is actually more of a chance that lighter players will. It looks like it's related to attempts more than anything. High risk, high reward.

## Analysis 4: Which teams pay players the most?

In [ ]:
salariesDf.head()

In [ ]:
teamsDf.head()

In [ ]:
teamsDf.name.unique()

That is a lot of teams. Let's just look at teams since 2010

In [ ]:
teams10 = teamsDf[teamsDf.yearID >= 2010 ]
salaries10 = salariesDf[salariesDf.yearID >= 2010]

In [ ]:
teams10.name.unique()

In [ ]:
salaries10 = pd.merge(salaries10, teams10[["teamID", "yearID", "name", "W", "R", "AB", "H", "DivWin",
                                          "WCWin", "LgWin", "WSWin"]], on = ["yearID", "teamID"])

In [ ]:
salaries10.head()

In [ ]:
plt.figure(figsize = (10,10))

# Get order
medSal = salaries10[["name", "salary"]].groupby("name").median().sort_values(by = ["salary"], ascending = False)
order = medSal.index

sns.boxplot(data = salaries10, x ="salary", y =  "name", fliersize=0, order = order)
plt.xlabel("Player Salary", fontdict = label)
plt.ylabel("Team", fontdict = label)
plt.title("Salary Distribution by Team Since 2010", fontdict = title)
plt.savefig('figs/SalaryByTeam.pdf', bbox_inches='tight')

Do teams that win the world series pay their players more?

In [ ]:
wonWS = salaries10[salaries10.WCWin == "Y"]
noWS = salaries10[salaries10.WCWin == "N"]

plt.figure(figsize = (8,6))
sns.kdeplot(data = wonWS, x ="salary", label = "Won World Series")
sns.kdeplot(data = noWS, x ="salary", label = "Did Not Win World Series")
plt.xlabel("Player Salary", fontdict = label)
plt.ylabel("Density", fontdict = label)
plt.xlim(-5000000, 20000000)
plt.legend()
plt.title("Teams that Did and Did Not Win the World Series", fontdict = title)
plt.savefig('figs/SalaryByWS.pdf')

What about any winner?

In [ ]:
# Create a column to show if they won anything
conditions = [
    ((salaries10.DivWin == "Y") | (salaries10.WCWin == "Y") | (salaries10.LgWin == "Y") | (salaries10.WSWin == "Y")),
    ((salaries10.DivWin != "Y") & (salaries10.WCWin != "Y") & (salaries10.LgWin != "Y") & (salaries10.WSWin != "Y"))
]
# tasks corresponding to above conditions
result = ["Y", "N"]
salaries10["AnyWin"] = np.select(conditions, result)

In [ ]:
salaries10.head()

In [ ]:
anyWin = salaries10[salaries10.AnyWin == "Y"]
noWin = salaries10[salaries10.AnyWin == "N"]

plt.figure(figsize = (8,6))
sns.kdeplot(data = anyWin, x ="salary", label = "Any Win")
sns.kdeplot(data = noWin, x ="salary", label = "No Wins")
plt.xlabel("Player Salary", fontdict = label)
plt.ylabel("Density", fontdict = label)
plt.xlim(-5000000, 20000000)
plt.legend()
plt.title("Salary Distributions of Any/No Wins", fontdict = title)
plt.savefig('figs/SalaryByAnyWin.pdf')

In [ ]:
# Show both distributions in the same chart
fig, axis = plt.subplots(ncols = 2)
sns.kdeplot(data = wonWS, x ="salary", label = "Won World Series", ax = axis[0])
sns.kdeplot(data = noWS, x ="salary", label = "Did Not Win World Series", ax = axis[0])
axis[0].legend()

sns.kdeplot(data = anyWin, x ="salary", label = "Any Win", ax = axis[1])
sns.kdeplot(data = noWin, x ="salary", label = "No Wins", ax = axis[1])
axis[1].legend()

fig.set_figheight(6)
fig.set_figwidth(14)

# Show t test results
display(f'Independent T-test (World Series): {scipy.stats.ttest_ind(wonWS.salary, noWS.salary)}')
display(f'Independent T-test (Any Win): {scipy.stats.ttest_ind(anyWin.salary, noWin.salary)}')